<h2 align ="center">Notebook with a server that connects to web pages and sends the prediction for the lung xrays through majority voting on KNN,Neural Networks,SVM,Random Forests</h2>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#installing dependencies
!pip install fastapi
!pip install pyngrok
!pip install nest-asyncio
!pip install uvicorn
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5ec14cd98ea8da20b115eb3c76af7c5bfa07fb76885d62395e5e3e26aa130851
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import numpy as np
from PIL import Image
import pickle

In [ ]:
from fastapi import FastAPI
import json
from pyngrok import ngrok
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
import io
import uvicorn
import base64
from fastapi import FastAPI, File, UploadFile, Request
from fastapi.responses import JSONResponse

<h2 align ="center">Importing all the models as well as compressors and scalers</h2>

In [15]:

with open('/content/drive/MyDrive/Data/scaler_model.pkl', 'rb') as f:
    scaler_model = pickle.load(f)

In [14]:
with open('/content/drive/MyDrive/Data/pca_model.pkl', 'rb') as f:
    compressor = pickle.load(f)

In [13]:
with open('/content/drive/MyDrive/Data/svm_model.pkl', 'rb') as f:
    SVM = pickle.load(f)

In [9]:
with open('/content/drive/MyDrive/Data/knn_model.pkl', 'rb') as f:
    KNN = pickle.load(f)

In [3]:
with open('/content/drive/MyDrive/Data/NN.pkl', 'rb') as f:
    NN = pickle.load(f)

In [16]:
with open('/content/drive/MyDrive/Data/random_forest.pkl', 'rb') as f:
    RF = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Data/random_forest_pca.pkl', 'rb') as f:
    RF_pca = pickle.load(f)
selected_features = np.load('/content/drive/MyDrive/Data/selected_features.npy')

In [17]:
#random forest predictor
def Rf_predicion(arr):
  a4 = RF_pca.transform(arr)
  a5 = a4[:, selected_features]
  return RF.predict(a5)

<h2 align ="center">Starting the server</h2>

In [ ]:
#api
app = FastAPI()

In [ ]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
@app.post("/")
async def index(request: Request):
    
    data = await request.json()
    image_data = data['image']
    image_data = image_data.split(',')[1]
    image_bytes = base64.b64decode(image_data)
    image = Image.open(io.BytesIO(image_bytes))
    # image_resized = image.resize((89401, 1)) # Resize to 89401 * 1 pixels
    
    image_array = np.array(image)
    image_flattened = image_array.flatten() # Flatten the image into a 1D array
    
    print('IMAGE: ',image_flattened)
    #image_flattened is the array u need to use Itqan, its the array
    
    a2 = scaler_model.transform([image_flattened])
    a3 = compressor.transform(a2)
    a = SVM.predict(a3)
    b = KNN.predict(a3)
    c = NN.predict(a3)
    d = Rf_predicion(a3)
    c = 2*c
    majority = np.zeros_like(a)
    #majority voting
    majority[(a+b+c+d)>=3] = 1
    print("Neural: ",a)
    print("KNN: ",b)
    print("SVM: ",c)
    print("RF: ",d)
    if majority[0] == 1:
        print("Positive")
        return JSONResponse({'result': True})
    else:
        print("Negative")
        return JSONResponse({'result': False})



    # Return the prediction as JSON
    # if prediction == 0:
    #     return JSONResponse({'result': 'healthy lungs'})
    # else:
    #     return JSONResponse({'result': 'COVID-19 lungs'})



In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)